# Семантичко језгро са интеграцијом OpenBnB MCP сервера

Овај нотебук показује како да користите Семантичко језгро са стварним OpenBnB MCP сервером за претрагу стварних Airbnb смештаја користећи MCPStdioPlugin. За приступ LLM-у користи Azure AI Foundry. Да бисте подесили променљиве окружења, можете пратити [лекцију за подешавање](/00-course-setup/README.md)


## Увези потребне пакете


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Креирање MCP прикључка

Повезаћемо се са [OpenBnB MCP сервером](https://github.com/openbnb-org/mcp-server-airbnb) користећи MCPStdioPlugin. Овај сервер пружа функционалност претраге за Airbnb преко пакета @openbnb/mcp-server-airbnb.


## Креирање клијента

У овом примеру ћемо користити Azure AI Foundry за приступ нашем LLM-у. Уверите се да су ваши променљиве окружења правилно подешене.


## Конфигурација окружења

Подесите Azure OpenAI поставке. Уверите се да су следеће променљиве окружења подешене:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Разумевање интеграције OpenBnB MCP

Овај нотебук се повезује са **правим OpenBnB MCP сервером** који пружа стварну функционалност претраге на Airbnb-у.

### Како функционише:

1. **MCPStdioPlugin**: Користи комуникацију преко стандардног улаза/излаза са MCP сервером
2. **Прави NPM пакет**: Преузима и покреће `@openbnb/mcp-server-airbnb` преко npx-а
3. **Живи подаци**: Враћа стварне податке о некретнинама са Airbnb API-ја
4. **Откривање функција**: Агент аутоматски открива доступне функције са MCP сервера

### Доступне функције:

OpenBnB MCP сервер обично пружа:
- **search_listings** - Претрага некретнина на Airbnb-у по локацији и критеријумима
- **get_listing_details** - Добијање детаљних информација о одређеним некретнинама
- **check_availability** - Провера доступности за одређене датуме
- **get_reviews** - Преузимање рецензија за некретнине
- **get_host_info** - Добијање информација о домаћинима некретнина

### Предуслови:

- **Node.js** инсталиран на вашем систему
- **Интернет конекција** за преузимање MCP сервер пакета
- **NPX** доступан (долази уз Node.js)

### Тестирање везе:

Можете ручно тестирати MCP сервер покретањем:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Ово ће преузети и покренути OpenBnB MCP сервер, на који се Semantic Kernel затим повезује ради приступа стварним подацима са Airbnb-а.


## Покретање агента са OpenBnB MCP сервером

Сада ћемо покренути AI агента који се повезује са OpenBnB MCP сервером како би тражио стварне Airbnb смештаје у Стокхолму за 2 одрасле особе и 1 дете. Слободно измените листу `user_inputs` да бисте променили критеријуме претраге.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Резиме
Честитамо! Успешно сте изградили AI агента који се интегрише са претрагом смештаја у стварном свету користећи Model Context Protocol (MCP):

## Коришћене технологије:
- Semantic Kernel - За изградњу интелигентних агената са Azure OpenAI
- Azure AI Foundry - За LLM могућности и завршетак разговора
- MCP (Model Context Protocol) - За стандардизовану интеграцију алата
- OpenBnB MCP сервер - За стварну функционалност претраге Airbnb смештаја
- Node.js/NPX - За покретање спољног MCP сервера

## Шта сте научили:
- MCP интеграција: Повезивање Semantic Kernel агената са спољним MCP серверима
- Приступ подацима у реалном времену: Претраживање стварних Airbnb некретнина преко живих API-ја
- Комуникација преко протокола: Коришћење stdio комуникације између агента и MCP сервера
- Откривање функција: Аутоматско откривање доступних функција са MCP сервера
- Стримовање одговора: Хватање и бележење позива функција у реалном времену
- HTML рендеровање: Форматирање одговора агента са стилизованим табелама и интерактивним приказима

## Следећи кораци:
- Интеграција додатних MCP сервера (време, летови, ресторани)
- Изградња система са више агената који комбинује MCP и A2A протоколе
- Креирање прилагођених MCP сервера за ваше изворе података
- Имплементација трајног памћења разговора кроз сесије
- Деплој агента на Azure Functions са оркестрацијом MCP сервера
- Додавање аутентификације корисника и могућности резервације



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако тежимо тачности, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква неспоразумевања или погрешна тумачења која могу произаћи из коришћења овог превода.
